In [ ]:
#Custom
from nyt_article_search import JSONParse
from spark_job_functions import *
#Spark
from pyspark.sql import SparkSession
from pyspark.sql import Window
from pyspark.sql import functions as f
from pyspark.sql.types import *
#Normal stuff
from itertools import chain
import pandas as pd
import json
import os

### Load JSONs and transform in to lists of tuples

In [ ]:
# Lists that the data from each json response will be added to
article_ids = []
fact_data = []
author_data = []
subject_data = []
people_data = []
org_data = []
loc_data = []
# Big text will be a dictionary where we'll add headline, lead paragraph,
# abstract and web url
big_text = {}

In [ ]:
data_folder = 'DATA'
state = 'TEXAS'
# states = [f for f in os.listdir(data_folder) if f != '_Archive']
state_folder = os.path.join(data_folder, state)
json_files = os.listdir(state_folder)
for file in json_files:
    filepath = os.path.join(state_folder, file)
    json_file = open(filepath, 'r')
    data = json.load(json_file)
    responses = data['response']['docs']
    # Parse each json response in the json file
    for respsonse in responses:
        j = JSONParse(respsonse)
        article_ids.append(j.article_id)
        fact_data.append(j.get_article_facts())
        extend_list(author_data, j.get_article_authors())
        extend_list(subject_data, j.search_article_keywords('subject'))
        extend_list(people_data, j.search_article_keywords('persons'))
        extend_list(org_data, j.search_article_keywords('organizations'))
        extend_list(loc_data, j.search_article_keywords('glocations'))
        id_text_dict = {
            j.article_id:{
                            'headline':j.get_text('headline')[1],
                            'abstract':j.get_text('abstract')[1],
                            'lead_paragraph':j.get_text('lead_paragraph')[1],
                            'web_url':j.get_text('web_url')[1]
                        }
            }
        big_text.update(id_text_dict)

In [ ]:
spark = SparkSession.\
        builder.\
        appName('NYT_JSON_ETL').\
        master('local[1]').\
        getOrCreate()

In [ ]:
#Create spark dfs
facts = spark.createDataFrame(fact_data, schema=get_table_headers('facts'))
authors = spark.createDataFrame(author_data, schema=get_table_headers('authors'))
subjects = spark.createDataFrame(subject_data, schema=get_table_headers('subjects'))
people = spark.createDataFrame(people_data, schema=get_table_headers('subjects'))
organizations = spark.createDataFrame(org_data, schema=get_table_headers('subjects'))
locations = spark.createDataFrame(loc_data, schema=get_table_headers('subjects'))

### Step 1: Create Primary Keys in tables

In [ ]:
# First step is to create an interger primary key for the article_ids
ids = [(e + 1000, i) for e, i in enumerate(article_ids)]
# Create spark df out of id list
# Integer Primary Key is fact_id
id_schema = StructType([
    StructField('fact_id', IntegerType(), False),
    StructField('article_id', StringType(), False)
    ])

### THIS SHOULD NOT CHANGE ANY FURTHER
id_df = spark.createDataFrame(ids, schema=id_schema)



#Merge the id_df dataframe into the existing frames
# To put fact_id in all the other tables
facts = id_df.join(facts, ['article_id'], how = 'inner').drop('article_id')
authors = id_df.join(authors, ['article_id'], how = 'inner').drop('article_id')
subjects = id_df.join(subjects, ['article_id'], how = 'inner').drop('article_id')
people = id_df.join(people, ['article_id'], how = 'inner').drop('article_id')
organizations = id_df.join(organizations, ['article_id'], how = 'inner').drop('article_id')
locations = id_df.join(locations, ['article_id'], how = 'inner').drop('article_id')

In [ ]:
# Union the subjects, organizations, and locations dfs togehter
places_and_things = subjects.union(organizations).union(locations).orderBy(['fact_id', 'rank'])

places_and_things = create_primary_key(places_and_things, 'table_id', 'fact_id', 'rank')
authors = create_primary_key(authors, 'table_id', 'fact_id', 'rank')
people = create_primary_key(people, 'table_id', 'fact_id', 'rank')

### Step 2: Text standardization

In [ ]:
# Read in dimensional news desk, section name, and article type files
dim_folder = os.path.join('DATA', 'DIM_TABLES')
dim_news_desk = os.path.join(dim_folder, 'news_desks.csv')
dim_types = os.path.join(dim_folder, 'article_types.csv')
dim_sections = os.path.join(dim_folder, 'section_names.csv')

news_desk_df = spark.read.option('header', True).csv(dim_news_desk)
material_df = spark.read.option('header', True).csv(dim_types)
section_df = spark.read.option('header', True).csv(dim_sections)

In [ ]:
#standardize any text in the article_type, news_desk, and section_name
#columns that aren't in the dimensinal tables
#EX: National Desk vs National - should be National
facts = standardize_text(facts, 'article_type', material_df)
facts = standardize_text(facts, 'news_desk', news_desk_df)
facts = standardize_text(facts, 'section_name', section_df)
facts.show(5)


In [ ]:
#clean the print_page column in facts df
#Some print pages aren't numeric: 25A - A is already in print_section column
#Create a list of unique page numbers 
page_rdd = facts.select(['print_page']).distinct().rdd.flatMap(lambda x: x).collect()
#include pages that aren't None
pages = [p for p in set(page_rdd) if p != None]

pages_cleaned = {}
for page in pages:
    if page != None:
        try:
            p = float(page)
            p_dict = {page:p}
            pages_cleaned.update(p_dict)
        except:
            p = page[:-1]
            p = float(p)
            p_dict = {page:p}
            pages_cleaned.update(p_dict)
    else:
        pass
#create mapping
mapping = f.create_map([f.lit(x) for x in chain(*pages_cleaned.items())])

#update the print_page column with mapping
facts = facts.withColumn('print_page', mapping[facts['print_page']])
facts.show(5)


In [ ]:
# IN the people df create a standardized first name, middle name, and last name columns

# Get the last name (left of comma) and upper case it
people = people.withColumn('last_name', f.upper(f.substring_index('value', ',', 1)))
# Get the first and middle name (right of comma) and upper case it
people = people.withColumn('first_middle', f.upper(f.substring_index('value', ',', -1)))
# Replace special characters in first and middle names with empty string,
# But keep inner white space to separate middle name from first
people = people.withColumn('first_middle', f.trim(f.regexp_replace('first_middle', '[^a-zA-Z ]', '')))
# Identify name qualifiers - SR, JR, II, III, IV, V
# EX - Dale Earnhardt vs Dale Earnhardt Jr
qualifier_list = ['JR', 'SR', 'II', 'III', 'IV', 'V']
qualifiers = people.where(f.substring_index('first_middle', ' ', -1).isin(qualifier_list)).select(['table_id', 'first_middle'])

#Map replacement string for names with ending qualifiers
name_replace = qualifiers.select('first_middle')
name_list = name_replace.select(['first_middle']).distinct().rdd.flatMap(lambda x: x).collect()
name_dict = {name:name[0:name.rindex(' ')] for name in name_list}

#update name_dict to include rest of names so mapping is correct
non_qualifiers = people.where(~f.substring_index('first_middle', ' ', -1).isin(qualifier_list)).select(['table_id', 'first_middle'])
non_qual_list = non_qualifiers.select(['first_middle']).distinct().rdd.flatMap(lambda x: x).collect()
for n in non_qual_list:
    name_dict.update({n:n})


#create mapping
name_map = f.create_map([f.lit(x) for x in chain(*name_dict.items())])
people = people.withColumn('first_middle', name_map[people['first_middle']])

#Get middle name / initial
people = people.withColumn('middle', 
    f.when(f.size(f.split(people['first_middle'], ' ', -1)) == 1, None).otherwise(f.substring_index(people['first_middle'], ' ', -1)))

#Get first name
people = people.withColumn('first_name', f.substring_index(people['first_middle'], ' ', 1))

#Merge qualifiers back in as a separate column
qualifiers = qualifiers.withColumn('qualifier', f.substring_index('first_middle', ' ', -1)).select('table_id', 'qualifier')
people = people.join(qualifiers, ['table_id'], how = 'left').drop('value').drop('first_middle')


In [ ]:
# Authors - rename columns to match people df
rename_authors_cols = {
    'lastname':'last_name',
    'firstname':'first_name',
    'middlename':'middle', 
    'rank':'role_rank',
    'role':'author_role'
}
authors = authors.withColumnsRenamed(rename_authors_cols)

#uppercase names
authors = authors.withColumn('last_name', f.upper('last_name'))
authors = authors.withColumn('first_name', f.upper('first_name'))
authors = authors.withColumn('middle', f.upper('middle'))


#people and plances_and_things - update columns
peole_col_renamed = {
    'rank':'subject_rank',
    'major':'major_subject'
}
people = people.withColumnsRenamed(peole_col_renamed)

places_and_things_col_renamed = {
    'rank':'subject_rank',
    'major':'major_subject',
    'value':'subject'
}
places_and_things = places_and_things.withColumn('value', f.upper('value'))
places_and_things = places_and_things.withColumnsRenamed(places_and_things_col_renamed)

#Make major_role column true or false
people = people.withColumn('major_subject', f.when(people['major_subject'] == 'Y', True).otherwise(False))
places_and_things = places_and_things.withColumn('major_subject', f.when(places_and_things['major_subject'] == 'Y', True).otherwise(False))


#### Step 3: Use Dimensional Id's instead of text

- Dimensional Tables:
    - article_type_ids: material_df
    - news_desk_ids: news_desk_df
    - section_ids: section_df
    - subjects_ids: create df

In [ ]:
#Make Sure Id's are ints
material_df = material_df.withColumn('article_type_id', material_df['article_type_id'].cast(IntegerType()))
news_desk_df = news_desk_df.withColumn('news_desk_id', news_desk_df['news_desk_id'].cast(IntegerType()))
section_df = section_df.withColumn('section_name_id', section_df['section_name_id'].cast(IntegerType()))

In [ ]:

facts = facts.join(material_df, ['article_type'], how = 'left')\
        .join(news_desk_df, ['news_desk'], how = 'left')\
        .join(section_df, ['section_name'], how = 'left')\
        .drop(*('section_name', 'news_desk', 'article_type'))


In [ ]:
subject_ids = [
    (1, 'persons'),
    (2, 'organizations'),
    (3, 'glocations'),
    (4, 'subject')
]
subject_id_df = spark.createDataFrame(subject_ids, schema=['subject_id', 'name'])

people = people.join(subject_id_df, ['name'], how = 'left').drop('name')
places_and_things = places_and_things.join(subject_id_df, ['name'], how = 'left').drop('name')
subject_id_df = subject_id_df.withColumnRenamed('name', 'subject_name')

In [ ]:
facts = facts.select(final_col_order('facts'))
authors = authors.select(final_col_order('authors'))
people = people.select(final_col_order('subject_people'))
places_and_things = places_and_things.select(final_col_order('subject_others'))

In [ ]:
#Validate Schemas
facts = spark.createDataFrame(facts.rdd.collect(), schema=StructType(schema_validation('facts')))
authors = spark.createDataFrame(authors.rdd.collect(), schema=StructType(schema_validation('authors')))
people = spark.createDataFrame(people.rdd.collect(), schema=StructType(schema_validation('subject_people')))
places_and_things = spark.createDataFrame(places_and_things.rdd.collect(), schema=StructType(schema_validation('subject_others')))
id_df = spark.createDataFrame(id_df.rdd.collect(), schema=StructType(schema_validation('article_ids')))